### Non-API workflow (deprecated)

This workflow only would work for Architecture v1 (see README).

- [] Export/retrieve survey answers as excel (no JSON support)
- [] Cloud Function: 
    * *Drop file into cloud storage*
    * *Trigger Serverless function*
        * Read excel file 
        * Translate to career onestop JSON format 
        * Send JSON in request to career onestop
        * Retrieve results from career onestop 
        * Send email to user provided address

In [ ]:
#### "Exporting" Survey Response (Create Mock Excel File)

Kamran took the survey I created. I can't actually export his answers because that's a paid feature but based on the [documentation](https://help.surveymonkey.com/en/surveymonkey/analyze/xls-exports/) we have a good idea of what the exported xlsx (only option besides PDF...) will look like. I have to manually export his response values from this...

![img](img/sm_survey_example_q.png)


In [ ]:
## Making mock export xlsx, manually entering his answers into excel sheet

# Get the content of the export from documentation
import pandas as pd 
doc_table = pd.read_html("https://help.surveymonkey.com/en/surveymonkey/analyze/xls-exports/")[0]

df = doc_table \
    .transpose() \
    .iloc[:,1:]
df.columns = df.iloc[0,:] 

df.drop(index=0, axis=0, inplace=True)

display(df.transpose())

# Making up time data to paste into the excel sheet as well
import datetime as dt 
start = dt.datetime.now()
start_str = dt.datetime.strftime(start, "%Y-%m-%d %X")
end_str = dt.datetime.strftime(start + dt.timedelta(minutes=5), "%Y-%m-%d %X")
start_str, end_str

In [ ]:
# After manually copy-pasting into excel file:
df = pd.read_excel('data/mock_export.xlsx').iloc[:,1:]
survey_answers = df.iloc[:,4:]
survey_answers


In [ ]:
# Translate our answer dictionary to the below format 
with open(os.path.join("data","get-skills-api","response.json"), "r") as file:
  co_survey_questions = json.load(file)['Skills']
co_survey_questions

In [ ]:
import os
import json 
import random

with open(os.path.join("data", "skills-submit-api", "example-request.json"), "r") as file:
  example_request = json.load(file)
print(len(example_request['SKAValueList']))

# Fill out our survey with rest of questions/answers (40 in total for Skills Matcher;  SM limited by free trial)
random_answers = [random.randint(1,5) for _ in range(30)]
random_answers_df = pd.DataFrame({f"Question {n+11}": [random_answers[n]] for n in range(30)}) 
random_answers_df

# join to Kamran's 
survey_answers_added = pd.concat([survey_answers, random_answers_df], axis=1)
survey_answers_added

In [ ]:
## Shouldn't assume pandas will work in a serverless function (it's a large dependency)
from openpyxl import load_workbook

def excel_to_dict(filename, random_answers):

    ## Replace this in cloud version with reading from cloud storage bucket 
    workbook = load_workbook(filename)
    worksheet = workbook.worksheets[0]

        # check for unexpected file name 

    ## Reading logic/error catching will vary based on what SM export looks like
    cols = [col.value for col in list(worksheet.rows)[0]][1:] # first row is column
    rows = [row.value for row in list(worksheet.rows)[1]][1:]

        # check for nans, unexpected file structure, len(cols) != len(rows)
  
    # Obviously wouldn't actually extend random answers in production
    cols = list(cols) + [f"Question {n+11}" for n in range(30)]
    rows = list(rows) + random_answers

    # Return Dict 
    excel_dict = {"question_map":[]} # we don't need to translate respondent ID, name, collector id, start date, or end date  
    for n in range(len(cols)):
        if "question|?" not in cols[n].lower(): # could also 
            excel_dict[cols[n]] = rows[n]
        else: 
            excel_dict["question_map"].append({"qn":1, "sm_question":cols[n],"sm_answer":rows[n]}) 
        
    # return dict(zip(cols, rows))
    return excel_dict

data = excel_to_dict("data/mock_export.xlsx", [random.randint(1,5) for _ in range(30)])
data

In [ ]:

def translate_excel_dict(data):
    data_copy = data.copy()
    answer_map = { 
            1: "DataPoint20",
            2: "DataPoint35",
            3: "DataPoint50",
            4: "DataPoint65",
            5: "DataPoint80"
        }
    
    question_map = {
            "Question 1": "2.C.1.a",
            "Question 2": "2.C.4.d",
            "Question 3": "1.A.3.c.3",
            "Question 4": "2.C.3.d",
            "Question 5": "2.C.4.c",
            "Question 6": "2.C.1.b",
            "Question 7": "2.B.2.i",
            "Question 8": "2.C.3.a",
            "Question 9": "2.C.1.e",
            "Question 10": "2.C.1.c",
            "Question 11": "2.C.7.c",
            "Question 12": "2.C.7.b",
            "Question 13": "2.B.1.f",
            "Question 14": "2.B.1.e",
            "Question 15": "2.B.5.b",
            "Question 16": "2.A.1.e",
            "Question 17": "2.C.3.e",
            "Question 18": "2.C.5.a",
            "Question 19": "1.A.1.d.1",
            "Question 20": "2.A.2.d",
            "Question 21": "2.B.1.d",
            "Question 22": "2.B.3.a",
            "Question 23": "2.C.1.f",
            "Question 24": "2.C.2.a",
            "Question 25": "2.B.3.e",
            "Question 26": "2.C.4.e",
            "Question 27": "2.C.8.a",
            "Question 28": "2.B.3.m",
            "Question 29": "2.B.3.l",
            "Question 30": "2.C.1.d",
            "Question 31": "2.A.1.f",
            "Question 32": "2.C.4.f",
            "Question 33": "2.A.1.d",
            "Question 34": "2.B.4.g",
            "Question 35": "2.C.6",
            "Question 36": "2.C.9.a",
            "Question 37": "2.C.5.b",
            "Question 38": "2.B.5.a",
            "Question 39": "2.B.3.k",
            "Question 40": "2.A.1.c"
        }
    

    # See the excel dict in the above cell to better understand this part  
    for sm_question, co_question in question_map.items():
        # Swap answers 
        sm_answer = data_copy[sm_question] # get old SM answer from input excel_dict
        co_answer = answer_map[sm_answer] # use map to translate to new CO readable answer

        # Swap questions
        data_copy[co_question] = data_copy.pop(sm_question) # .pop(key) 1.) removes old question:answer pair 2.) returns old key value   
        data_copy[co_question] = 

        
        
    return data